# Extracted Lesion Segmentation Visualizer

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from ipywidgets import interactive, widgets
from IPython.display import display

In [ ]:
# Function to plot an image from a list at a specified index
def plot_image(list_of_images, list_of_masks, image_idx, mask_transparency):
    original_image = np.rot90(list_of_images[image_idx], k=3)
    mask = np.rot90(list_of_masks[image_idx], k=3)

    # Overlay the mask on the original image
    masked_image = np.ma.masked_where(mask == 0, original_image)
    
    plt.imshow(original_image, cmap='gray')
    plt.imshow(masked_image, cmap='jet', alpha=mask_transparency)  # Adjust alpha for transparency
    plt.axis('off')  # Turn off axis numbers
    plt.show()

# Function to get a transparency scale widget
def get_transparency_scale():
    return widgets.FloatSlider(
        value=0.5,
        min=0,
        max=1,
        step=0.1,
        description='Mask Transparency:',
        continuous_update=False
    )

# Function to get a slider widget
def get_slider(max_value):
    return widgets.IntSlider(
        value=0,
        min=0,
        max=max_value,
        step=1,
        description='Image Index:',
        continuous_update=False
    )

# Wrapper function to create an interactive plot with a list of images
def create_interactive_plot(list_of_images, list_of_masks, title):
    slider = get_slider(len(list_of_images) - 1)
    transparency_scale = get_transparency_scale()
    interactive_plot = interactive(plot_image, list_of_images=widgets.fixed(list_of_images), list_of_masks=widgets.fixed(list_of_masks), 
                                    mask_transparency=transparency_scale, image_idx=slider)
    # Display the title as a text above the plot
    display(widgets.Label(value=f'\n{title}', style={'description_width': 'initial'}))
    display(interactive_plot)

def plot_interactive_plot(original_path, mask_path, patient_number):
    original_series_files = sorted(
        [file for file in os.listdir(os.path.join(original_path, f'{patient_number}')) 
         if file.endswith('0000.nii.gz') or file.endswith('0001.nii.gz') or file.endswith('0002.nii.gz') ]
    )
    all_files = os.listdir(os.path.join(original_path, f'{patient_number}'))
    mask_file = list(set(all_files) - set(original_series_files))
    common_mask_file = os.path.join(mask_path, f'{patient_number}',mask_file[0])

    # Load the common mask file
    common_mask_image = nib.load(common_mask_file).get_fdata()

    for original_file in original_series_files:
        original_file_path = os.path.join(original_path, f'{patient_number}', original_file)

        original_image = nib.load(original_file_path).get_fdata()
        original_images_to_slide_over = [original_image[:, :, i] for i in range(original_image.shape[2])]

        # Use the common mask for all original images
        mask_images_to_slide_over = [common_mask_image[:, :, i] for i in range(common_mask_image.shape[2])]

        # Get the series name from the original file (you may need to adjust this depending on your file naming convention)
        series_name = os.path.splitext(original_file)[0]

        create_interactive_plot(original_images_to_slide_over, mask_images_to_slide_over, series_name)


For private data visualization.

In [ ]:
patient = "154"
original_path = f'/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Private_Extracted_1_Slice'
mask_path = f'/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Private_Extracted_1_Slice'
plot_interactive_plot(original_path, mask_path, patient)

For Picai data visualization.

In [ ]:
patient = "10005"
original_path = f'/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Picai_All_Extracted_3_Slice'
mask_path = f'/local_ssd/practical_wise24/prostate_cancer/NNUNet_Lesion/Picai_All_Extracted_3_Slice'
plot_interactive_plot(original_path, mask_path, patient)